In [38]:
# initial imports

import h5py
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pickle
import pandas as pd

import torchvision
import torch
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights
import torch.optim as optim


import matplotlib as mpl
from matplotlib import pyplot as plt

In [39]:
# Add path to and import mkturk analysis tools

from sys import path

print(path)

path.append('/Users/parsatalaie/Desktop/Issa Data')

from data_analysis_tools_mkTurk.utils_meta import get_recording_path
from data_analysis_tools_mkTurk.general import df_2_psth_mat
from data_analysis_tools_mkTurk.IO import ch_dicts_2_h5, h5_2_trial_df, h5_2_df

['/Users/parsatalaie/Desktop/Marmoset Data Analysis', '/opt/anaconda3/lib/python312.zip', '/opt/anaconda3/lib/python3.12', '/opt/anaconda3/lib/python3.12/lib-dynload', '', '/opt/anaconda3/lib/python3.12/site-packages', '/opt/anaconda3/lib/python3.12/site-packages/aeosa']


/Users/parsatalaie/Desktop/Issa Data/data_analysis_tools_mkTurk/general.py:22: UserWarning: Failed to import analysis_metadata module.
  warnings.warn('Failed to import analysis_metadata module.')
/Users/parsatalaie/Desktop/Issa Data/data_analysis_tools_mkTurk/IO.py:19: UserWarning: Failed to import analysis_metadata module.
  warnings.warn('Failed to import analysis_metadata module.')


In [40]:
import os

h5_paths = []

for root, dirs, files in os.walk('/Volumes/issa-locker/users/Dan/ephys/Bourgeois'):
    for file in files:
        if file.endswith('.h5'):
             h5_paths.append(os.path.join(root, file))

print(h5_paths[:20])

['/Volumes/issa-locker/users/Dan/ephys/Bourgeois/Bourgeois_20250130_R_H00_P55/20250130.h5', '/Volumes/issa-locker/users/Dan/ephys/Bourgeois/Bourgeois_20241115_R_H00_P14/20241115.h5', '/Volumes/issa-locker/users/Dan/ephys/Bourgeois/Bourgeois_20241206_R_H00_P27/20241206.h5', '/Volumes/issa-locker/users/Dan/ephys/Bourgeois/Bourgeois_20250117_R_H00_P48/20250117.h5', '/Volumes/issa-locker/users/Dan/ephys/Bourgeois/Bourgeois_20241226_R_H00_P37/20241226.h5', '/Volumes/issa-locker/users/Dan/ephys/Bourgeois/Bourgeois_20250103_R_H00_P41/20250103.h5', '/Volumes/issa-locker/users/Dan/ephys/Bourgeois/Bourgeois_20241203_R_H00_P24/20241203.h5', '/Volumes/issa-locker/users/Dan/ephys/Bourgeois/Bourgeois_20250311_R_H01_P21/20250311.h5', '/Volumes/issa-locker/users/Dan/ephys/Bourgeois/Bourgeois_20241202_R_H00_P23/20241202.h5', '/Volumes/issa-locker/users/Dan/ephys/Bourgeois/Bourgeois_20241210_R_H00_P28/20241210.h5', '/Volumes/issa-locker/users/Dan/ephys/Bourgeois/Bourgeois_20250331_R_H01_P31/20250331.h5'

In [41]:
# HvM
scenefiles = ['/mkturkfiles/scenebags/West/hvm10_table_45_20240906.json',
       '/mkturkfiles/scenebags/West/hvm10_elephant_45_20240906.json',
       '/mkturkfiles/scenebags/West/hvm10_dog_45_20240906.json',
       '/mkturkfiles/scenebags/West/hvm10_bear_45_20240906.json',
       '/mkturkfiles/scenebags/West/hvm10_chair_45_20240906.json',
       '/mkturkfiles/scenebags/West/hvm10_car_45_20240906.json',
       '/mkturkfiles/scenebags/West/hvm10_turtle_45_20240906.json',
       '/mkturkfiles/scenebags/West/hvm10_plane_45_20240906.json',
       '/mkturkfiles/scenebags/West/hvm10_apple_45_20240906.json',
       '/mkturkfiles/scenebags/West/hvm10_head_45_20240906.json']

single_scenefile = '/mkturkfiles/scenebags/West/hvm10_table_45_20240906.json'

In [42]:
hvm_paths = []

for path in h5_paths:
    trial_params = h5_2_trial_df(path)
    if single_scenefile in trial_params.scenefile.values:
        hvm_paths.append(path)

print(hvm_paths[:10])

/opt/anaconda3/lib/python3.12/site-packages/tables/attributeset.py:290: DataTypeWarning: Unsupported type for attribute 'scenefile_by_stim_mat' in node '/'. Offending HDF5 class: 8
  value = self._g_getattr(self._v_node, name)
/opt/anaconda3/lib/python3.12/site-packages/tables/attributeset.py:290: DataTypeWarning: Unsupported type for attribute 'scenefile_by_stim_mat' in node '/'. Offending HDF5 class: 8
  value = self._g_getattr(self._v_node, name)
/opt/anaconda3/lib/python3.12/site-packages/tables/attributeset.py:290: DataTypeWarning: Unsupported type for attribute 'scenefile_by_stim_mat' in node '/'. Offending HDF5 class: 8
  value = self._g_getattr(self._v_node, name)
/opt/anaconda3/lib/python3.12/site-packages/tables/attributeset.py:290: DataTypeWarning: Unsupported type for attribute 'scenefile_by_stim_mat' in node '/'. Offending HDF5 class: 8
  value = self._g_getattr(self._v_node, name)
/opt/anaconda3/lib/python3.12/site-packages/tables/attributeset.py:290: DataTypeWarning: Uns

['/Volumes/issa-locker/users/Dan/ephys/Bourgeois/Bourgeois_20241206_R_H00_P27/20241206.h5', '/Volumes/issa-locker/users/Dan/ephys/Bourgeois/Bourgeois_20241202_R_H00_P23/20241202.h5', '/Volumes/issa-locker/users/Dan/ephys/Bourgeois/Bourgeois_20241227_R_H00_P38/20241227.h5', '/Volumes/issa-locker/users/Dan/ephys/Bourgeois/Bourgeois_20241126_R_H00_P21/20241126.h5', '/Volumes/issa-locker/users/Dan/ephys/Bourgeois/Bourgeois_20241127_R_H00_P22/20241127.h5']


In [44]:
hvm_paths

['/Volumes/issa-locker/users/Dan/ephys/Bourgeois/Bourgeois_20241206_R_H00_P27/20241206.h5',
 '/Volumes/issa-locker/users/Dan/ephys/Bourgeois/Bourgeois_20241202_R_H00_P23/20241202.h5',
 '/Volumes/issa-locker/users/Dan/ephys/Bourgeois/Bourgeois_20241227_R_H00_P38/20241227.h5',
 '/Volumes/issa-locker/users/Dan/ephys/Bourgeois/Bourgeois_20241126_R_H00_P21/20241126.h5',
 '/Volumes/issa-locker/users/Dan/ephys/Bourgeois/Bourgeois_20241127_R_H00_P22/20241127.h5']